# [NOME DO PROJETO] - Análise Exploratória de Dados

**Objetivo:** Descreva aqui o objetivo principal desta análise.

**Fonte dos Dados:** Indique a origem dos dados (ex: Sistema de Vendas, Relatório de Marketing, etc.).

**Data:** 17 de agosto de 2025

## 1. Configuração do Ambiente

Nesta seção, importamos as bibliotecas e configuramos o ambiente para uma melhor visualização dos dados.

In [ ]:
# Importação de bibliotecas essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importação das nossas ferramentas
from ferramentas_analista import carregar_dados, converter_notebook_para_py

# --- Configurações de Visualização (Essencial para Análise) ---
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None) # Descomente se precisar ver todas as linhas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Configurações de gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Ambiente configurado com sucesso!")

## 2. Carga de Dados

Utilizamos a função `carregar_dados` para importar os dados de forma inteligente (CSV, Excel, JSON, etc.).

In [ ]:
#caminho_arquivo_dados = 'caminho/para/seu/arquivo.csv'
caminho_arquivo_dados = 'tad.csv' 
df = carregar_dados(caminho_arquivo_dados)

if df is not None:
    df_analise = df.copy()

## 3. Inspeção Inicial dos Dados (Dossiê do Dataset)

Esta é a etapa mais crucial para 'entender' os dados. Realizamos uma série de verificações básicas para criar um dossiê completo sobre o dataset que recebemos.

In [ ]:
if 'df_analise' in locals() and df_analise is not None:
    print(f"O dataset contém {df_analise.shape[0]} linhas e {df_analise.shape[1]} colunas.\n")
    
    print("--- 1. Visualização das Primeiras e Últimas Linhas ---")
    display(df_analise.head())
    display(df_analise.tail())

    print("\n--- 2. Nomes de Todas as Colunas ---")
    print(df_analise.columns.tolist())
    
    print("\n--- 3. Tipos de Dados e Uso de Memória ---")
    df_analise.info(memory_usage='deep')
    
    print("\n--- 4. Análise de Linhas Duplicadas (em colunas seguras) ---")
    # Identifica colunas que não podem ser usadas na verificação (que contêm listas, por exemplo)
    colunas_problematicas = []
    for col in df_analise.columns:
        if df_analise[col].apply(lambda x: isinstance(x, (list, dict))).any():
            colunas_problematicas.append(col)
    
    if colunas_problematicas:
        print(f"Aviso: As seguintes colunas contêm listas ou dicts e serão ignoradas na checagem de duplicatas: {colunas_problematicas}")
    
    # Verifica duplicatas apenas nas colunas que não são problemáticas
    colunas_para_verificar = [col for col in df_analise.columns if col not in colunas_problematicas]
    num_duplicatas = df_analise.duplicated(subset=colunas_para_verificar).sum()
    
    if num_duplicatas > 0:
        print(f"\033[91mAVISO: Foram encontradas {num_duplicatas} linhas duplicadas (considerando apenas colunas simples).\033[0m")
    else:
        print("✅ Nenhuma linha duplicada encontrada (considerando apenas colunas simples).")
        
    print("\n--- 5. Análise de Valores Únicos (Cardinalidade) ---")
    print("Contagem de valores únicos por coluna (ajuda a identificar IDs e constantes):")
    # Apenas para colunas que não são problemáticas
    display(df_analise[colunas_para_verificar].nunique().sort_values(ascending=True).to_frame(name='Contagem de Únicos'))
    
    print("\n--- 6. Análise de Valores Nulos (Percentual) ---")
    percent_nulos = (df_analise.isnull().sum() / len(df_analise) * 100).sort_values(ascending=False)
    percent_nulos = percent_nulos[percent_nulos > 0]
    if not percent_nulos.empty:
        print("Colunas com valores nulos:")
        display(percent_nulos.to_frame(name='Percentual de Nulos (%)'))
    else:
        print("✅ Nenhuma coluna com valores nulos encontrada.")
        
    print("\n--- 7. Estatísticas Descritivas (Colunas Numéricas) ---")
    display(df_analise.describe())
    
    print("\n--- 8. Estatísticas Descritivas (Colunas Categóricas) ---")
    display(df_analise.describe(include=['object', 'category']))

## 4. Limpeza e Pré-Processamento (Checklist de ETL)

Com base no dossiê acima, tratamos os problemas identificados. Descomente e adapte os blocos de código que forem relevantes para o seu projeto.

In [ ]:
# --- 4.1 Seleção e Remoção de Colunas ---
# # Remover uma ou mais colunas irrelevantes
# colunas_para_remover = ['id_transacao', 'coluna_inutil']
# df_analise.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

# --- 4.2 Renomeação de Colunas ---
# # Renomear colunas específicas para nomes mais claros
# df_analise.rename(columns={'NM_CLIENTE': 'nome_cliente', 'DT_VENDA': 'data_venda'}, inplace=True)

# # Padronizar todos os nomes de colunas (ex: para minúsculas e snake_case)
# df_analise.columns = df_analise.columns.str.lower().str.replace(' ', '_', regex=False)

# --- 4.3 Tratamento de Dados Ausentes ---
# # Estratégia 1: Remover linhas onde colunas importantes são nulas
# df_analise.dropna(subset=['id_cliente', 'valor_compra'], inplace=True)

# # Estratégia 2: Preencher nulos com um valor específico (medidas de tendência central ou constantes)
# df_analise['idade'].fillna(df_analise['idade'].median(), inplace=True)
# df_analise['categoria_produto'].fillna('Desconhecida', inplace=True)

# --- 4.4 Correção de Tipos de Dados ---
# # Converter colunas de data que foram lidas como texto
# df_analise['data_venda'] = pd.to_datetime(df_analise['data_venda'])

# # Exemplo prático: Converter uma coluna de preço (ex: 'R$ 1.234,56') para número
# df_analise['preco'] = df_analise['preco'].astype(str).str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# # Converter para tipos de dados que economizam memória
# df_analise['status'] = df_analise['status'].astype('category')

# --- 4.5 Tratamento de Duplicatas ---
# # Remover linhas que são inteiramente duplicadas (usando o subset seguro definido na inspeção)
# df_analise.drop_duplicates(subset=colunas_para_verificar, inplace=True)

# --- 4.6 Criação de Novas Colunas (Engenharia de Features) ---
# # Extrair informações de datas
# df_analise['ano_venda'] = df_analise['data_venda'].dt.year
# df_analise['mes_venda'] = df_analise['data_venda'].dt.month

# # Criar categorias a partir de uma variável numérica (Binning)
# df_analise['faixa_etaria'] = pd.cut(df_analise['idade'], bins=[0, 18, 35, 60, 100], labels=['Jovem', 'Adulto', 'Meia-Idade', 'Idoso'])

# --- 4.7 Limpeza e Padronização de Strings ---
# # Remover espaços em branco no início e no fim
# df_analise['nome_produto'] = df_analise['nome_produto'].str.strip()

# # Padronizar para letras minúsculas
# df_analise['cidade'] = df_analise['cidade'].str.lower()

print("Etapa de limpeza e pré-processamento concluída. Verifique os resultados abaixo.")
# df_analise.info() # É uma boa prática verificar os tipos de dados novamente

## 5. Análise Exploratória de Dados (EDA)

Com os dados limpos, investigamos e visualizamos os dados para encontrar padrões e insights.

### 5.1 Análise Univariada
Análise de cada variável individualmente para entender sua distribuição e características.

In [ ]:
# --- Para variáveis NUMÉRICAS --- 
# SUGESTÃO: Use Histogramas para ver a distribuição e Boxplots para identificar outliers.

# # Exemplo com uma coluna numérica (descomente e adapte)
# coluna_numerica = 'sua_coluna_numerica'
# sns.histplot(data=df_analise, x=coluna_numerica, kde=True).set_title(f'Distribuição de {coluna_numerica}')
# plt.show()

# sns.boxplot(data=df_analise, x=coluna_numerica).set_title(f'Boxplot de {coluna_numerica}')
# plt.show()

In [ ]:
# --- Para variáveis CATEGÓRICAS ---
# SUGESTÃO: Use Gráficos de Barras para ver a frequência de cada categoria.

# # Exemplo com uma coluna categórica (descomente e adapte)
# coluna_categorica = 'sua_coluna_categorica'
# sns.countplot(data=df_analise, y=coluna_categorica, order = df_analise[coluna_categorica].value_counts().index).set_title(f'Contagem de {coluna_categorica}')
# plt.show()

### 5.2 Análise Bivariada
Análise da relação entre pares de variáveis.

In [ ]:
# --- Relação entre duas variáveis NUMÉRICAS ---
# SUGESTÃO: Use Gráfico de Dispersão (Scatter Plot) para identificar correlações.

# # Exemplo (descomente e adapte)
# sns.scatterplot(data=df_analise, x='coluna_numerica_X', y='coluna_numerica_Y').set_title('Relação entre X e Y')
# plt.show()

In [ ]:
# --- Relação entre uma variável NUMÉRICA e uma CATEGÓRICA ---
# SUGESTÃO: Use Boxplots ou Violin Plots para comparar a distribuição da variável numérica entre as categorias.

# # Exemplo (descomente e adapte)
# sns.boxplot(data=df_analise, x='coluna_categorica', y='coluna_numerica').set_title('Distribuição Numérica por Categoria')
# plt.xticks(rotation=45)
# plt.show()

In [ ]:
# --- Relação entre duas variáveis CATEGÓRICAS ---
# SUGESTÃO: Use uma Tabela de Contingência (crosstab) e um Mapa de Calor (Heatmap) para visualizar a relação.

# # Exemplo (descomente e adapte)
# contingency_table = pd.crosstab(df_analise['coluna_categorica_1'], df_analise['coluna_categorica_2'])
# sns.heatmap(contingency_table, annot=True, fmt='d', cmap='YlGnBu')
# plt.title('Mapa de Calor da Relação entre Categoria 1 e 2')
# plt.show()

## 6. Conclusões e Próximos Passos

In [ ]:
print("Escreva aqui os principais insights e próximos passos da análise.")

## 7. Exportação do Script

Convertemos este notebook em um script Python limpo.

In [ ]:
# Certifique-se de que o nome do arquivo corresponde ao seu notebook
converter_notebook_para_py('notebook_modelo.ipynb')